<a href="https://colab.research.google.com/github/wanjiku-wairimu1/Social-media-analysis-helper-API-/blob/main/Copy_of_social_network_analysis_helper.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/SimPPL/SocialMediaAnalysis.git

remote: Enumerating objects: 8, done.
remote: Counting objects: 100% (8/8), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 5 (delta 2), reused 5 (delta 2), pack-reused 0
Unpacking objects: 100% (5/5), done.
From https://github.com/SimPPL/SocialMediaAnalysis
 * branch            HEAD       -> FETCH_HEAD
Updating 8160f64..f2c56b1
Fast-forward
 .../Collect Tweets via the API-checkpoint.ipynb    | 2588 +-------------------
 ...entiment Analysis with RoBERTa-checkpoint.ipynb |   96 +-
 Collect Tweets via the API.ipynb                   | 2588 +-------------------
 Sentiment Analysis with RoBERTa.ipynb              |   96 +-
 4 files changed, 34 insertions(+), 5334 deletions(-)


In [ ]:
# the bad way of doing things
api_key = ''
api_key_secret = ''
bearer_token = ''
access_token = ''
access_token_secret = ''

In [ ]:
!pip install --upgrade tweepy pandas numpy plotly tqdm

In [ ]:
import tweepy
import pandas as pd
import numpy as np
import time
import glob
import json                          
import plotly.graph_objects as go    
import tqdm
from tqdm import trange
from tqdm import tqdm
import datetime as dt
import requests

In [ ]:
client = tweepy.Client(bearer_token=bearer_token)

In [ ]:
search_url = "https://api.twitter.com/2/tweets/search/recent"

In [ ]:
news_outlets = '((url:"www.bloomberg.com") OR (url:"www.fortune.com") OR (url:"www.theguardian.com") OR (url:"www.npr.org") OR (url:"www.salon.com") OR  (url:"www.newsweek.com") OR (url:"www.politico.com") OR (url:"www.pbs.org") OR (url:"www.economist.com") OR (url:"www.nbc.com") OR (url:"www.abcnews.com") OR (url:"www.cbsnews.com") OR (url:"www.cnn.com") OR (url:"www.foxnews.com") OR (url:"www.sfchronicle.com"))'

In [ ]:
def bearer_oauth(r):
    '''Construct an authentication header for the search service 
    '''
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

In [ ]:
# documentation: https://developer.twitter.com/en/docs/twitter-api/tweets/search/integrate/build-a-query#build
query_params = {'query': news_outlets + ' -is:retweet',
           'tweet.fields': 'author_id,entities', 'max_results':100}


In [ ]:
def connect_to_endpoint(url, params):
    '''Send the Twitter API a request to get back some data on the query you built.
    '''
    # this is called a GET request
    response = requests.get(url, auth=bearer_oauth, params=params)
    
    if response.status_code != 200:  # code 200 indicates OK
        raise Exception(response.status_code, response.text)
    return response.json()

In [ ]:
response = connect_to_endpoint(search_url, query_params)

In [ ]:
response['meta']

{'newest_id': '1598353709259526146',
 'oldest_id': '1598353295273529345',
 'result_count': 100,
 'next_token': 'b26v89c19zqg8o3fpzhml1ukhzsbsgmn0oicmsblg00al'}

In [ ]:
# This is necessary because the API generates results in pages.
def paginate(r):
    token = r['meta']['next_token']
    new_params = query_params
    new_params.update({'pagination_token':token})
    return connect_to_endpoint(search_url, new_params)


In [ ]:
responses = [response]

In [ ]:
# Output an array given a search query:
def parse_response(res):
    if 'urls' in res['entities']:
        return ['www.twitter.com/anyuser/status/' + str(res['id']), res['text'], res['entities']['urls'][0]['expanded_url']]
    else:
        return ['www.twitter.com/anyuser/status/' + str(res['id']), res['text'], '']


In [ ]:
# We arbitrarily chose to use 10 pages of max length 100.
for i in range(10):
    responses.append(paginate(responses[i]))

# Save all the responses to a .csv file:
table = []
def add_to_table(res):
    for x in range(res['meta']['result_count']):
        row = []
        row += (parse_response(((res)['data'])[x]))
        table.append(row)

for r in responses:
    add_to_table(r)

df = pd.DataFrame({'Link': [i[0] for i in table], 
                   'Body': [i[1] for i in table],
                   'Article': [i[2] for i in table],
                   })
                   
df.to_csv('news_tweets.csv')


In [ ]:
!pip install --upgrade transformers scipy pandas plotly ekphrasis numpy torch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached transformers-4.24.0-py3-none-any.whl (5.5 MB)
     |████████████████████████████████| 33.8 MB 1.4 MB/s 
  Using cached ekphrasis-0.5.4-py3-none-any.whl (83 kB)
     |██████████████████████████████  | 834.1 MB 1.3 MB/s eta 0:00:42tcmalloc: large alloc 1147494400 bytes == 0x39e72000 @  0x7fe9e0e1c615 0x5d631c 0x51e4f1 0x51e67b 0x4f7585 0x49ca7c 0x4fdff5 0x49caa1 0x4fdff5 0x49ced5 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x4f60a9 0x55f926 0x5d7c18 0x5d9412 0x586636 0x5d813c 0x55f3fd 0x55e571 0x5d7cf1 0x49ced5 0x55e571 0x5d7cf1 0x49ec69 0x5d7c18 0x49ca7c 0x4fdff5 0x49ced5
     |████████████████████████████████| 890.2 MB 4.2 kB/s 
     |████████████████████████████████| 7.6 MB 19.3 MB/s 
  Using cached huggingface_hub-0.11.1-py3-none-any.whl (182 kB)
     |████████████████████████████████| 52 kB 1.2 MB/s 
     |████████████████████████████████| 53 kB 1.6 MB/s 
     |█████████████████

In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer
import numpy as np
from scipy.special import softmax
import csv
import urllib.request
import pandas as pd
import plotly
from plotly import graph_objects as go
import plotly.express as px
import ekphrasis
from ekphrasis.classes.preprocessor import TextPreProcessor
from ekphrasis.classes.tokenizer import SocialTokenizer
from ekphrasis.dicts.emoticons import emoticons
from tqdm import tqdm

In [ ]:
def preprocess(text):
    '''This function takes in text and removes links and usernames from it.
    Parameters
    ----------
    text : string of unprocessed text (contains links and usernames)
    '''
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)

In [ ]:
# You can run alternate versions of this model specialized to different tasks
# emoji, emotion, hate, irony, offensive, sentiment
task='sentiment'
MODEL = f"cardiffnlp/twitter-roberta-base-{task}"
tokenizer = AutoTokenizer.from_pretrained(MODEL)


Downloading:   0%|          | 0.00/747 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

In [ ]:
# download label mapping
labels=[]
mapping_link = f"https://raw.githubusercontent.com/cardiffnlp/tweeteval/main/datasets/{task}/mapping.txt"
with urllib.request.urlopen(mapping_link) as f:
    html = f.read().decode('utf-8').split("\n")
    csvreader = csv.reader(html, delimiter='\t')
labels = [row[1] for row in csvreader if len(row) > 1]


In [ ]:
# Load pretrained Pytorch model
model = AutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
tokenizer.save_pretrained(MODEL)

Downloading:   0%|          | 0.00/499M [00:00<?, ?B/s]

('cardiffnlp/twitter-roberta-base-sentiment/tokenizer_config.json',
 'cardiffnlp/twitter-roberta-base-sentiment/special_tokens_map.json',
 'cardiffnlp/twitter-roberta-base-sentiment/vocab.json',
 'cardiffnlp/twitter-roberta-base-sentiment/merges.txt',
 'cardiffnlp/twitter-roberta-base-sentiment/added_tokens.json',
 'cardiffnlp/twitter-roberta-base-sentiment/tokenizer.json')

In [ ]:
df = pd.read_csv('news_tweets.csv')

In [ ]:
def analyze(text, model):
    '''Take in text and the model as inputs and returns a score
    '''
    processed_text = preprocess(text)
    encoded_input = tokenizer(processed_text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    return scores

In [ ]:
# For Twitter, where the .csv files are already seperated by categories:
outputs=[]                        
labels = {0: 'Negative', 1:'Neutral', 2:'Positive'}

ct = 0

for i in tqdm(df['Body']):
    ct += 1
    res = analyze(i, model)
    outputs.append([i, res])
    if ct > 5:
      break
# From this, we obtained all the information we used in our study about the sentiment of tweets


  0%|          | 5/2096 [00:01<12:11,  2.86it/s]


In [ ]:
outputs


[['@RossTuckerNFL How long should an accompaniment take to prepare? https://t.co/PVVQ9KcKmh',
  array([0.13728379, 0.8352483 , 0.02746786], dtype=float32)],
 ["Student loans relief: 5th Circuit Court rejects Biden's latest plea to reinstate program | Fox News https://t.co/pPRyV2OwY1",
  array([0.22684473, 0.71926093, 0.05389433], dtype=float32)],
 ["0 yeah we splurge on things like bread, milk and gasoline....\nJanet Yellen blames Americans' 'splurging' for record-high #inflation\n\nhttps://t.co/wbEapCIlML",
  array([0.6733569 , 0.30129072, 0.02535237], dtype=float32)],
 ["Thank you, @m_scribe for continuing to advocate for children. If this won't convince you to #MaskUp, what will? #ProtectionOverPolitics https://t.co/XxntoyIkLm",
  array([0.04582128, 0.2876314 , 0.6665473 ], dtype=float32)],
 ['Approximately 20% of our water bills each year pay dividends and interest payments, the Competition &amp; Markets authority found. Privatised utility companies are rinsing us. Re-nationalise t

In [ ]:
model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerN